In [31]:
import json
with open('Data_Embedding/test.json', 'r') as f:
    x = json.load(f)
    
with open('test.json', 'r') as f:
    y = json.load(f)
x_stars = set()
x_2 = set()
y_stars = set()
for i in range(len(x)):
    x_stars.add(x[i]['text'])
    x_2.add(x[i]['stars'])
for i in range(len(y)):
    y_stars.add(y[i]['text'])
print(len(x_stars - x_stars.union(y_stars)))

0


In [32]:
print(len(x))

800


In [38]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def visualize_hyperparameter_results(grid_search_results, output_dir):
    """
    Create visualization of hyperparameter tuning results
    """
    # plt.style.use('seaborn')
    
    # Split results by learning rate
    lr_groups = {0.1: [], 0.01: []}
    for result in grid_search_results:
        lr_groups[result['learning_rate']].append(result)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    for lr, ax in zip([0.1, 0.01], [ax1, ax2]):
        results = lr_groups[lr]
        
        # Create scatter points for each (hidden_dim, dropout) pair
        hidden_dims = sorted(set(r['hidden_dim'] for r in results))
        dropouts = sorted(set(r['dropout'] for r in results))
        
        # Create color map for dropout values
        colors = plt.cm.viridis(np.linspace(0, 1, len(dropouts)))
        
        for i, dropout in enumerate(dropouts):
            x_vals = []  # hidden dims
            y_vals = []  # val accuracies
            sizes = []   # epochs (for bubble size)
            
            for hd in hidden_dims:
                result = next(r for r in results 
                            if r['hidden_dim'] == hd and r['dropout'] == dropout)
                x_vals.append(hd)
                y_vals.append(result['best_val_acc'])
                sizes.append(result['best_epoch'] * 20 + 100)  # Scale for visibility
            
            ax.scatter(x_vals, y_vals, s=sizes, c=[colors[i]], 
                      alpha=0.6, label=f'Dropout={dropout}')
            
            # Add connecting lines
            ax.plot(x_vals, y_vals, '--', c=colors[i], alpha=0.3)
        
        ax.set_xlabel('Hidden Dimension')
        ax.set_ylabel('Best Validation Accuracy')
        ax.set_title(f'Learning Rate = {lr}')
        ax.grid(True, alpha=0.3)
        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        
        # Annotate points with epochs
        for result in results:
            ax.annotate(f"{result['best_epoch']+1}",
                       (result['hidden_dim'], result['best_val_acc']),
                       xytext=(5, 5), textcoords='offset points',
                       fontsize=8)
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'hyperparameter_comparison.png'), 
                bbox_inches='tight', dpi=300)
    plt.close()

    # Create heatmap visualization
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    for lr, ax in zip([0.1, 0.01], [ax1, ax2]):
        results = lr_groups[lr]
        
        # Create matrix for heatmap
        acc_matrix = np.zeros((len(dropouts), len(hidden_dims)))
        for i, dropout in enumerate(dropouts):
            for j, hd in enumerate(hidden_dims):
                result = next(r for r in results 
                            if r['hidden_dim'] == hd and r['dropout'] == dropout)
                acc_matrix[i, j] = result['best_val_acc']
        
        sns.heatmap(acc_matrix, annot=True, fmt='.3f', 
                    xticklabels=hidden_dims, yticklabels=dropouts,
                    ax=ax, cmap='YlOrRd')
        ax.set_xlabel('Hidden Dimension')
        ax.set_ylabel('Dropout')
        ax.set_title(f'Validation Accuracy (LR={lr})')
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'hyperparameter_heatmap.png'), 
                bbox_inches='tight', dpi=300)
    plt.close()
    


In [39]:
# Load CSV data and prepare grid search results
def load_data(csv_file_path):
    # Load the CSV file into a DataFrame
    df = pd.read_csv(csv_file_path)
    # Convert DataFrame rows to a list of dictionaries (each row is a dictionary)
    grid_search_results = df.to_dict(orient='records')
    return grid_search_results

# Define output directory
output_dir = 'results/plots'
os.makedirs(output_dir, exist_ok=True)

# Load data and visualize
csv_file_path = 'results/grid_search_results.csv'  # Path to your CSV file
grid_search_results = load_data(csv_file_path)

# Call the visualization function
visualize_hyperparameter_results(grid_search_results, output_dir)

In [46]:
import pickle
word_embedding = pickle.load(open('./Data_Embedding/word_embedding.pkl', 'rb'))
print(len(word_embedding['the']))
!pwd

50
/Users/shubhampatel/Desktop/UTD/Sem 4 F24/CS 6320/Assignments/Assignment 2/assignment2_release
